In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from keras_model import build_model

In [2]:
df = pd.read_csv('..\\data\\processed2\\processed_data.csv', index_col=0)
df_games = pd.read_csv('..\\data\\processed2\\final_dataset_games.csv', index_col=1)
df_games.drop("Unnamed: 0", axis=1, inplace=True)

In [3]:
n_batters = len(df['res batter'].unique())
n_pitchers = len(df['res pitcher'].unique())

In [4]:
df['event_final'] = df['event_final'].astype('category')

In [5]:
df['event_final_code'] = df['event_final'].cat.codes

In [6]:
first = [0]*len(df)
second = [0]*len(df)
third = [0]*len(df)
for i in df.index:
    if isinstance(df.loc[i, '1st runner'], str):
        first[i] = 1
    if isinstance(df.loc[i, '2nd runner'], str):
        second[i] = 1
    if isinstance(df.loc[i, '3rd runner'], str):
        third[i] = 1
    if i % 1000 == 0:
        print(i)
first = np.array(first)
second = np.array(second)
third = np.array(third)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [7]:
df['first'] = first
df['second'] = second
df['third'] = third

In [8]:
situation = df[['inning', 'batting team', 'outs', 'visiting_score', 'home_score', 'first', 'second', 'third']].values
batter = df['res batter'].astype('category').cat.codes.values
pitcher = df['res pitcher'].astype('category').cat.codes.values
balls = df['balls'].values
strikes = df['strikes'].values
fouls = df['fouls'].values
outcome = df['event_final_code'].values
outcome_onehot = pd.get_dummies(df['event_final_code']).values

In [9]:
situation[:, 0] -= 1

In [10]:
cats = df['event_final'].astype('category')
bats = df['res batter'].astype('category')
pits = df['res pitcher'].astype('category')

outcome_dict = {}
for i, cat in enumerate(cats.cat.categories):
    outcome_dict[cat] = i
outcome_dict_rev = dict(enumerate(cats.cat.categories))
N_outcomes = len(outcome_dict)

bat_dict = {}
for i, bat in enumerate(bats.cat.categories):
    bat_dict[cat] = i
bat_dict_rev = dict(enumerate(bats.cat.categories))

pit_dict = {}
for i, pit in enumerate(pits.cat.categories):
    pit_dict[cat] = i
pit_dict_rev = dict(enumerate(pits.cat.categories))

In [11]:
ev_order=df['event_final'].value_counts().index
order = []
for ev in ev_order:
    order += [outcome_dict[ev]]

In [12]:
test = range(839996, 903996)

In [13]:
df = df.loc[test].iloc[:-24]

In [14]:
game_groups = df.groupby('gameid')

In [15]:
games = list(game_groups.groups.keys())

In [16]:
def update_situation(situation, outcome, model):
    vals = np.array(list(situation.values()))[3:].astype(np.float32)
    pred = model([vals.reshape(1, -1), outcome.astype(np.int32)])
    pred = [np.squeeze(p.numpy()) for p in pred]
    valid = False
    count = 0
    while not valid and count < 20:
        count += 1
        inning = np.random.choice(2, p=[1-pred[0], pred[0]])
        half_inning = np.random.choice(2, p=[1-pred[1], pred[1]])
        outs = np.random.choice(pred[2].shape[0], p=pred[2])
        visiting_score = np.random.choice(pred[3].shape[0], p=pred[3])
        home_score = np.random.choice(pred[4].shape[0], p=pred[4])
        first = np.random.choice(2, p=[1-pred[5], pred[5]])
        second = np.random.choice(2, p=[1-pred[6], pred[6]])
        third = np.random.choice(2, p=[1-pred[7], pred[7]])
        situation_out = {'inning diff' : inning,
                         'home score diff' : home_score,
                         'visiting score diff' : visiting_score,
                         'half inning' : half_inning,
                         'outs' : outs,
                         'first' : first,
                         'second' : second,
                         'third' : third}
        valid = compare_situation(situation, situation_out)
        if not valid:
            print('reject')
    return situation_out, pred

def compare_situation(situation, update):
    base_runners = situation['first'] + situation['second'] + situation['third']
    base_runners_update = update['first'] + update['second'] + update['third']
    base_runners_diff = base_runners_update - base_runners
    if update['half inning'] != situation['half inning']:
        if update['outs'] != 0:
            print('reset outs')
            return False
        elif (not update['half inning']) & (update['inning diff'] != 1):
            print('reset inning')
            return False
        elif update['first'] or update['second'] or update['third']:
            print('reset base runners')
            return False
        else:
            return True
    elif (update['half inning'] == situation['half inning']) & (update['outs'] < situation['outs']):
        print('outs decreasing')
        return False
    elif update['half inning'] & \
        (update['outs'] - situation['outs'] + update['home score diff'] - 1 != -base_runners_diff):
        print('more outs than base runner change')
        return False
    elif (not update['half inning']) & \
        (update['outs'] - situation['outs'] + update['visiting score diff'] - 1 != -base_runners_diff):
        print('more outs than base runner change')
        return False
    else:
        return True

def get_sit_feed(sit, out, outcome_dict):
    sit_feed = {'inning' : sit[0],
                'home_score' : sit[1], 
                'visiting_score' : sit[2],
                'half inning' : sit[3],
                'outs' : sit[4],
                'first' : sit[5],
                'second' : sit[6],
                'third' : sit[7]}
    return sit_feed, outcome_dict[out]

def get_sit_out_test(y_test, ind):
    test = [t[ind] for t in y_test]
    sit_out = {'inning_diff' : int(round(test[0])),
               'home score diff' : np.where(test[4])[0][0],
               'visiting score diff' : np.where(test[3])[0][0],
               'half inning' : int(round(test[1])),
               'outs' : np.where(test[2])[0][0],
               'first' : int(round(test[5])),
               'second' : int(round(test[6])),
               'third' : int(round(test[7]))}
    return sit_out

In [17]:
def get_lineups(game_df):
    home_pitcher = game_df.loc['home starting pitcher']
    visit_pitcher = game_df.loc['visiting starting pitcher']
    home_lineup = game_df.loc[[c for c in game_df.index if 'home batter' in c]]
    visitor_lineup = game_df.loc[[c for c in game_df.index if 'visitor batter' in c]]
    return home_pitcher, visit_pitcher, home_lineup.values, visitor_lineup.values

In [19]:
pred_model = build_model( n_batters, n_pitchers, 8, 4, 12, 99, 1 )
pred_model.load_weights('..\\models\\prediction_model_v0')

In [20]:
state_start = np.load('..\\models\\prediction_model_v0_state.npy')
pred_model.layers[9].reset_states(states=np.expand_dims(state_start, axis=0))